# From Targets to Fiber Assigment - Files and Databases 


In [27]:
import numpy as np
import yaml
from astropy.table import Table
from astropy.io import fits
import desispec.maskbits as masks
import desimodel.io

# 1. YAML target description

Describes the different kinds of targets, bit offsets, its priorities and number of required observations.

In [28]:
!cat ../../desidata/inputs/targets.yaml

targetmask:
    - [QSOLYA,       0, "Lyman alpha QSO", "DARK", 1, 5]
    - [QSO,       1, "Tracer QSO", "DARK", 1, 1]
    - [LRG,      2, "Luminous Red Galaxy", "DARK", 3, 2]
    - [ELG, 3, "Emission Line Galaxy", "DARK", 5, 1]
    - [STAR,    4, "Standard Star", "ANY", 4, 1]
    - [SKY, 5, "Sky", "ANY", 2, 1] 
    - [BGS, 6, "Bright Galaxy", "BRIGHT", 1, 1]
# Name
# bitoffset
# comment
# survey
# priority (lower value indicated higher priority)
# number of observations




In [29]:
! cat ../../desidata/inputs/target_priorities.yaml

targetpriorities:
    - [QSOLYA, 100, "Lyman alpha QSO"]
    - [QSO, 200, "Tracer QSO"]
    - [LRG, 300, "Luminous Red Galaxy"]
    - [ELG, 400, "Emission Line Galaxy"]
    - [STAR, 500, "Standard Star"]
    - [SKY, 600, "Sky"]
    - [BGS_BRIGHT, 700, "r<19.5 gals without redshift measurement"]
    - [BGS_FAINT, 720, "19.5<r<20.0 gals without redshift measurement"]
    - [BGS_BRIGHT_ZCONFIRMED, 740, "BGS_BRIGHT with successful redshift measurement."]
    - [BGS_FAINT_ZCONFIRMED, 760, "BGS_FAINT with successful redshift measurement"]
    - [MW_STAR, 800, "MW star"]
# Name
# Priority
# Comment




We can use the code in desispec.maskbit to work with target mask

In [30]:
stream = open('../../desidata/inputs/targets.yaml', 'r')
_alldefs = yaml.load(stream)
stream.close()
attr = []
for line in _alldefs["targetmask"]:
    attr.append(line[:3])
_targetbitdefs = dict([("targetmask", attr)])
targetmask = masks.BitMask("targetmask", _targetbitdefs)

This is a set of random targets over 10 different tiles to give explicit examples about the formats and files.

In [31]:
n_tiles = 10
target_per_tile = 25000
total_n_targets = target_per_tile * n_tiles / 2
random_ID_list =  np.arange(total_n_targets, dtype='int64')
random_ra_list = np.random.random(total_n_targets)
random_dec_list = np.random.random(total_n_targets)
random_type_A_list = np.int_(np.random.random(total_n_targets)*7)
random_type_B_list = np.int_(np.random.random(total_n_targets)*7)

This is the set of fibers.

In [32]:
fibers = desimodel.io.load_fiberpos()
fiberid = fibers['FIBER']
n_fibers = np.size(fiberid)

# 2. DESI targets

In real life the targets should be created from either a database, tractor files or mock files.
Includes at least four columns

* Target ID (int)
* RA (float)
* DEC (float)
* Type (int64) Mask

In [33]:
def write_target_fits(id_list, ra_list, dec_list, type_list, filename):
    t = Table([id_list, ra_list, dec_list, type_list], names=('ID', 'RA', 'DEC', 'TYPE'), meta={'name': 'target table'})
    t.write(filename, format='fits', overwrite=True)
    
for i_tile in range(n_tiles):
    #choose randomly targets on different tiles
    filename = 'tmp/targets_%06d.fits'%(i_tile)
    tile_id = np.random.choice(random_ID_list, replace=False,size=target_per_tile)
    tile_ra = random_ra_list[tile_id]
    tile_dec = random_dec_list[tile_id]
    tile_type_A = random_type_A_list[tile_id]
    tile_type_B = random_type_B_list[tile_id] 
    tile_type = np.ones(target_per_tile, dtype='int64')
    
    #create mock types
    for i in range(target_per_tile):
        tile_type[i]  = targetmask.mask(random_type_A_list[i]) | targetmask.mask(random_type_B_list[i])
        
    #write to disk
    write_target_fits(tile_id, tile_ra, tile_dec, tile_type, filename)

# 3. DESI observations

Track the objects that had a fiber put on a given tile.
They contain at least two columns
* Object-id (int)
* ObservedFlag (int) Yes/No (1/0) flag.

In [34]:
def write_observations_fits(id_list, observed_list, filename):
    t = Table([id_list, observed_list], names=('ID', 'OBSFLAG'), meta={'name': 'observations table'})
    t.write(filename, format='fits', overwrite=True)

for i_tile in range(n_tiles):
    filename = 'tmp/targets_%06d.fits'%(i_tile)
    t = Table.read(filename, format='fits')
    
    object_id = t['ID']
    observed_id = np.random.choice(np.arange(np.size(object_id)), replace=False,size=n_fibers)
    observed_flag = np.zeros(np.size(object_id), dtype=int)
    observed_flag[observed_id] = 1
    
    filename = 'tmp/observations_%06d.fits'%(i_tile)
    write_observations_fits(object_id, observed_flag, filename)

# 4. DESI results

Track the results of processed observed tiles after redshift fitting
They contain at least three columns

* Object-id (int)
* DesiType (int64) Mask
* z_spec (float)

There is already a format proposed for this: zbest-BRICKNAME

https://github.com/desihub/desidatamodel/blob/master/doc/DESI_SPECTRO_REDUX/PRODNAME/bricks/BRICKNAME/zbest-BRICKNAME.rst

In [35]:
cat ../../desidatamodel/doc/DESI_SPECTRO_REDUX/PRODNAME/bricks/BRICKNAME/zbest-BRICKNAME.rst

zbest-BRICKNAME.fits

*This is a placeholder for the redshift data model*

This holds the classification and redshift information for targets.
The formats are TBD, but it should be row-matched to the spectra in
coadd-BRICKNAME.rst .  This is not yet the case.

Nominally the HDUs will be:

  - HDU0 (empty)
  - HDU1 (ZBEST) : binary table with best redshift fit results
  - ...

Inputs

Written by XXX, using:

  - coadd (coadded spectra)
  - brick (individual spectra)
  
HDU1
----

EXTNAME = ZBEST

*Current description of ZBEST; this will evolve*
  
Required Data Table Columns
~~~~~~~~~~~~~~~~~~~~~~~~~~~

========= ======== ===== ===========
Name      Type     Units Description
========= ======== ===== ===========
BRICKNAME char[8]        Brick name from targeting, e.g. 1234p567
TARGETID  int64          Unique target ID
Z         float64        Best fit redshift
ZERR      float64        Uncertainty on redshift
ZWARN     int64          Warning bitmask (TBD)
TYPE      char[20]       Object 

In [13]:
def write_results_fits(id_list, type_list, redshift_list, filename):
    t = Table([id_list, type_list, redshift_list], names=('ID', 'TYPE', 'REDSHIFT'), meta={'name': 'results table'})
    t.write(filename, format='fits', overwrite=True)

#generates mock results for each tile
for i_tile in range(n_tiles):
    target_file = 'tmp/targets_%06d.fits'%(i_tile)
    observations_file = 'tmp/observations_%06d.fits'%(i_tile)
    targets = Table.read(target_file, format='fits')
    observations = Table.read(observations_file, format='fits')
    
    
    targets_id = targets['ID']
    observations_id = observations['ID']
    n_targets = np.size(targets_id)
    
    assert np.size(targets_id)==np.size(observations_id)
    
    z_list = np.zeros(n_targets)
    type_list  = targets['TYPE'].copy()
    for i in range(n_targets):
        z_list[i] = -1.0
        type_list[i] = targets['TYPE'][i]
        if(observations['OBSFLAG'][i]==1):
            z_list[i] = np.random.random()
            type_bit_offset = np.int_(np.random.random()*7)
            type_list[i] = targetmask.mask(type_bit_offset)
    
    filename = 'tmp/results_%06d.fits'%(i_tile)
    write_results_fits(targets_id, type_list, z_list, filename)

# 5. Merged Target List

Uses YAML target description + targets/observations/results files to produce a single file to be fed into fiber assignment.

* Object-id (int)
* RA (float)
* DEC (float)
* FiberPriority (int)
* FiberObservationsNeeded (int)

In [14]:
def write_mtl_fits(id_list, ra_list, dec_list, priority_list, nobs_list, filename):
    t = Table([id_list, ra_list, dec_list, priority_list, nobs_list],
              names=('ID', 'RA', 'DEC', 'PRIOR', 'NOBS'), meta={'name': 'MTL table'})
    t.write(filename, format='fits', overwrite=True)
    
for i_tile in range(n_tiles):
    targets_filename = 'tmp/targets_%06d.fits'%(i_tile)
    observations_filename = 'tmp/observations_%06d.fits'%(i_tile)
    results_filename = 'tmp/results_%06d.fits'%(i_tile)
    mtl_filename = 'tmp/mtl_%06d.fits'%(i_tile)
    
    targets = Table.read(targets_filename, format='fits')
    observations = Table.read(observations_filename, format='fits')
    results = Table.read(results_filename, format='fits')

    
    n_points  = np.size(observations['ID'])
    
    mtl_type = np.ones(n_points, dtype='int64')
    mtl_priority = np.ones(n_points, dtype='int')
    mtl_nobs = np.ones(n_points, dtype='int')
    mtl_id = np.ones(n_points, dtype='int64')
    mtl_ra = np.ones(n_points)
    mtl_dec = np.ones(n_points)
    mtl_ra = targets['RA']
    mtl_dec = targets['DEC']
    mtl_id = targets['ID']
    for i in range(n_points):
        for name in targetmask.names():
            if((targets['TYPE'][i] & targetmask.mask(name)) != 0):
                break
        mtl_type[i] = targetmask.mask(name)
        mtl_priority[i] = _alldefs['targetmask'][targetmask.bitnum(name)][4]
        mtl_nobs[i] = _alldefs['targetmask'][targetmask.bitnum(name)][4] - observations['OBSFLAG'][i]
        
    write_mtl_fits(mtl_id, mtl_ra, mtl_dec, mtl_priority, mtl_nobs, mtl_filename)

# 6. Fiber assignment 


## 6.1 Available

A run of fiber assignment should output all the IDs that are available for each fiber on each tile. This allows us to correct for fiber assignment incompleteness and compare different fiber assignment algorithms.
Stores at least two columns

* Object-ID
* Fiber-ID

## 6.2 Assignment

A run of fiber assignment should output the final selection of fibers and targets on each tile.

Stores at least three columns

    Target-ID
    RA
    DEC
    Fiber-ID

These two tables are stored in the same FITS file

In [25]:
def table_available_fits(id_list, fiber_list, filename):
    t = Table([id_list, fiber_list],
              names=('ID', 'FIBERID'), meta={'name': 'Available table'})
    return t
    
def table_assign_fits(id_list, ra_list, dec_list, fiber_list, filename):
    t = Table([id_list, ra_list, dec_list, fiber_list],
              names=('ID', 'RA', 'DEC', 'FIBERID'), meta={'name': 'Assign table'})
    return t
    
#load fibers
fibers = desimodel.io.load_fiberpos()
fiberid = fibers['FIBER']
n_fibers = np.size(fiberid)

for i_tile in range(n_tiles):
    mtl_filename = 'tmp/mtl_%06d.fits'%(i_tile)
    assign_filename = 'tmp/assignment_%06d.fits'%(i_tile)
    available_filename = 'tmp/available_%06d.fits'%(i_tile)
    fiber_filename = 'tmp/fiberassign_%06.fits'%(i_tile)
    mtl = Table.read(mtl_filename, format='fits')

    # do the mock of available fibers
    available_id = np.empty((0))
    available_fiber = np.empty((0))
    for i in range(n_fibers):
        n_points = np.random.choice(np.arange(6))
        
        if(n_points!=0):
            selection = np.random.choice(np.arange(n_points), replace=False,size=n_points)
            available_id = np.append(available_id, selection)
            available_fiber = np.append(available_fiber, np.ones(n_points, dtype='int')*fiberid[i])
    
    #do the mock assignent
    n_points = np.size(mtl['ID'])
    selection = np.random.choice(np.arange(n_points), replace=False,size=n_fibers)
    assign_id  = mtl['ID'][selection]
    assign_ra  = mtl['RA'][selection]
    assign_dec = mtl['DEC'][selection]
    assign_fiber = fiberid

    T_available = table_available_fits(available_id, available_fiber, fiber_filename)
    T_assign = table_assign_fits(assign_id, assign_ra, assign_dec, assign_fiber, fiber_filename)
    
    #PENDING: How to write two tables in a single FITS file on different HDU
    T_available.write(fiber_filename, format='fits', overwrite=True)
    T_assign.write(fiber_filename, format='fits', overwrite=True)
    

In [17]:
def write_assign_fits(id_list, ra_list, dec_list, fiber_list, filename):
    t = Table([id_list, ra_list, dec_list, fiber_list],
              names=('ID', 'RA', 'DEC', 'FIBERID'), meta={'name': 'Assign table'})
    t.write(filename, format='fits', overwrite=True)

for i_tile in range(n_tiles):
    mtl_filename = 'tmp/mtl_%06d.fits'%(i_tile)
    assign_filename = 'tmp/assignment_%06d.fits'%(i_tile)

    mtl = Table.read(mtl_filename, format='fits')
    n_points = np.size(mtl['ID'])
    selection = np.random.choice(np.arange(n_points), replace=False,size=n_fibers)
    assign_id  = mtl['ID'][selection]
    assign_ra  = mtl['RA'][selection]
    assign_dec = mtl['DEC'][selection]
    assign_fiber = fiberid

    write_assign_fits(assign_id, assign_ra, assign_dec, assign_fiber, assign_filename)